In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

brownie networks add "Ethereum" "mainnet" host="https://eth-mainnet.alchemyapi.io/v2/0-R5velBm8shxaZfUraYwa8kCaBesQVH" 
brownie networks modify "mainnet" host="https://eth-mainnet.alchemyapi.io/v2/0-R5velBm8shxaZfUraYwa8kCaBesQVH" name="Ethereum Mainnet" chainid=1
brownie networks set_provider "alchemy"
Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
Error adding Ethereum blockchain to database Ethereum, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(Ethereum) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'Ethereum', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) skipping...
Error adding exchange carbon_v1 to database, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constr

C:\Users\Kveen\AppData\Local\Temp\ipykernel_8188\1582963263.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on MAINNET [NBTest021]

## MAINNET Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_MAINNET)
print( C.w3.provider.endpoint_uri)

# add_tenderly = f'brownie networks add "Ethereum" "{C.NETWORK_ID}" host="{C.provider_url}" name="{C.network_name}" chainid={C.CHAIN_ID}'
# mod_tenderly = f'brownie networks modify "{C.network_id}" host="{C.provider_url}" name="{C.network_name}" chainid={C.CHAIN_ID}'
# set_tenderly = f'brownie networks set_provider "{C.network_id}" host="{C.provider_url}" name="{C.network_name}" chainid={C.CHAIN_ID}'
#
# print(add_tenderly, mod_tenderly, set_tenderly)

assert C.DATABASE == C.DATABASE_POSTGRES
assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_MAINNET
assert C.PROVIDER == C.PROVIDER_ALCHEMY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

https://eth-mainnet.alchemyapi.io/v2/0-R5velBm8shxaZfUraYwa8kCaBesQVH


In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)

2023-05-09 17:02:32,078 [fastlane:ERROR] - [get_curves] error converting pool to curve PoolAndTokens(ConfigObj=Config(network=_ConfigNetworkMainnet(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderAlchemy()), id=413, cid='476', last_updated=datetime.datetime(2023, 5, 3, 18, 9, 46, 91747), last_updated_block=17218553, descr='uniswap_v3 BLOCKIFY-3C21/USDC-eB48 10000', pair_name='BLOCKIFY-3C21/USDC-eB48', exchange_name='uniswap_v3', fee=0.01, fee_float='0.01', tkn0_balance=None, tkn1_balance=None, z_0=0, y_0=0, A_0=0, B_0=0, z_1=0, y_1=0, A_1=0, B_1=0, sqrt_price_q96=Decimal('8225770990658924917'), tick=-459790, tick_spacing=200, liquidity=Decimal('57799302803353870547'), address='0xCbf08332BDE3ACEFE4cE77d74213c72b3885C1f0', anchor='NaN', tkn0='BLOCKIFY-3C21', tkn1='USDC-eB48', tkn0_address='0x7D608CfAa805a752788847e5C46Ce9C03Fb43C21', tkn0_decimals=18, tkn1_address='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', tkn1_decimals=6, tkn0_key='BLOCKIFY-3C21', tk

Number of curves: 2744
Number of tokens: 1772
Exchanges: {'sushiswap_v2', 'uniswap_v2', 'bancor_v3', 'carbon_v1', 'bancor_v2'}


In [4]:
bad_curves = ['340282366920938463463374607431768211527-0',
              '340282366920938463463374607431768211527-1',
              '12930729942995661611608235082407192035404-0',
              '9868188640707215440437863615521278132277-1',
                # '2381976568446569244243622252022377480199-0',
#               '3743106036130323098097120681749450326030-0',
#               '4423670769972200025023869896612986748950-0'
]
subcurves = [x for x in CCm if x.cid not in bad_curves]
CCm2 = CPCContainer([x for x in subcurves if x.params.exchange not in ['bancor_v2']])
len(CCm2)

2553

## Actual run

In [5]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [6]:
ops = bot._find_arbitrage_opportunities_carbon_single_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.AO_CANDIDATES)
ops

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj


[(Decimal('-0E-49'),
             DAI-1d0F  USDC-eB48
  839             0.0        0.0
  AMMIn           0.0        0.0
  AMMOut          0.0        0.0
  TOTAL NET       0.0        0.0,
  [{'cid': '839',
    'tknin': 'DAI-1d0F',
    'amtin': 0,
    'tknout': 'USDC-eB48',
    'amtout': 0,
    'error': None}],
  'USDC-eB48',
  (CPCArbOptimizer.TradeInstruction(cid='839', tknin='DAI-1d0F', amtin=0, tknout='USDC-eB48', amtout=0, error=None),)),
 (Decimal('-0E-49'),
                                    USDC-eB48  DAI-1d0F
  56ae8dd6f6e44916af9b037ade6b2675        0.0       0.0
  AMMIn                                   0.0       0.0
  AMMOut                                  0.0       0.0
  TOTAL NET                               0.0       0.0,
  [{'cid': '56ae8dd6f6e44916af9b037ade6b2675',
    'tknin': 'USDC-eB48',
    'amtin': 0,
    'tknout': 'DAI-1d0F',
    'amtout': 0,
    'error': None}],
  'USDC-eB48',
  (CPCArbOptimizer.TradeInstruction(cid='56ae8dd6f6e44916af9b037ade6b2675', tknin='U

In [7]:
# ord = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ORDSCAL)
# ord

In [8]:
# ti = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_TI)
# ti

In [9]:
# ti

In [10]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [11]:
candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
candf.reset_index(inplace=True, drop=True)
candf.iloc[0][2]

[{'cid': '803',
  'tknin': 'USDC-eB48',
  'amtin': 7790.291927469894,
  'tknout': 'WETH-6Cc2',
  'amtout': -4.234699934522723,
  'error': None},
 {'cid': '1701411834604692317316873037158841057353-0',
  'tknin': 'WETH-6Cc2',
  'amtin': 4.234699912369251,
  'tknout': 'USDC-eB48',
  'amtout': -7851.133636474609,
  'error': None}]

In [12]:
# print(CCm.bycid('340282366920938463463374607431768211527-1').description)

In [13]:
candf.iloc[0][0]

Decimal('129.760489434787978826209787750274157118283990808293992813560180366039276123047')

In [14]:
exact = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_EXACT)
exact

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj


exact trade amts: next_amount_in=7782.501635542424, amount_in=7782.501635, amount_out=4.217775872355509556
[_calc_carbon_output] before decode:  7851133629 7851133629 0 12119777774
[_calc_carbon_output] after decode:  7851.133629 7851.133629 0.0 43.058100281712086000000
exact trade amts: next_amount_in=4.217775872355509556, amount_in=4.217775872355509556, amount_out=7804.116953


[TradeInstruction(ConfigObj=Config(network=_ConfigNetworkMainnet(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderAlchemy()), cid='803', tknin='USDC-eB48', amtin=Decimal('7782.501635'), tknout='WETH-6Cc2', amtout=Decimal('4.217775872355509556'), pair_sorting='', raw_txs='[]', custom_data='', db=DatabaseManager(ConfigObj=Config(network=_ConfigNetworkMainnet(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderAlchemy()), contracts={}, filters=[], data=                                       cid                last_updated  \
 0     00125d26-4f9d-4936-9a46-7e7708cee9b5  2023-05-06 19:04:10.202420   
 1219  5bbd01d2-ba90-49e8-8235-0f5183a44204  2023-05-06 19:11:11.844129   
 1231  5c976f61-5847-4219-a2ee-0b1eb96b2775  2023-05-06 19:18:59.176213   
 1230  5c87da80-15a0-4c59-baac-e8e9661ee920  2023-05-06 19:00:47.931782   
 2635  c21eb1b9-cf04-4ec5-bad1-77fb1b29b38b  2023-05-06 19:13:37.076908   
 ...                              

In [15]:
exact[0].amtin_wei, exact[0].amtout_wei

(7782501635, 4217775872355509556)

In [16]:
exact[1].amtin_wei, exact[1].amtout_wei

(4217775872355509556, 7804116953)

In [17]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj


exact trade amts: next_amount_in=7782.501635542424, amount_in=7782.501635, amount_out=4.217775872355509556
[_calc_carbon_output] before decode:  7851133629 7851133629 0 12119777774
[_calc_carbon_output] after decode:  7851.133629 7851.133629 0.0 43.058100281712086000000
exact trade amts: next_amount_in=4.217775872355509556, amount_in=4.217775872355509556, amount_out=7804.116953
trade_instruction.cid: 803
trade_instruction.cid: 1701411834604692317316873037158841057353


In [18]:
route_struct, flashloan_amount, flashloan_token_address

([{'exchangeId': 5,
   'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
   'minTargetAmount': 1,
   'deadline': 1683647455,
   'customAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'customInt': 3000,
   'customData': '0x'},
  {'exchangeId': 6,
   'targetToken': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'minTargetAmount': 1,
   'deadline': 1683647455,
   'customAddress': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'customInt': 2000,
   'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000005000000000000000000000000000000490000000000000000000000000000000000000000000000003a888ccf25f6a534'}],
 7782501635,
 '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48')

# Tenderly Tx Submission

In [19]:
# C.w3.provider.endpoint_uri

In [20]:
# C.w3.eth.getBlock(C.w3.eth.block_number).timestamp

In [21]:
# C.provider.CARBON_CONTROLLER_CONTRACT.strategy(340282366920938463463374607431768211527)

In [22]:
### This to manually submit
# bot._validate_and_submit_transaction_tenderly(
#         ConfigObj = C,
#         route_struct = route_struct,
#         src_amount = flashloan_amount,
#         src_address = flashloan_token_address,
#             )

# MAINNNET Preferred Tx Submission Method

In [23]:
from fastlane_bot.helpers import TxHelpers

In [24]:


flashloan_amount = 1145804339352947836180
flashloan_token_address = "0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C"

pool = (
    bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
)
bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# Init TxHelpers
tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# Submit tx

current_gas_price, current_max_priority_gas, block_number, nonce = tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
                                                    src_address=flashloan_token_address, bnt_eth=bnt_eth,
                                                    expected_profit=Decimal('1000'), safety_override=True, result=tx_helpers.XS_API_CALLS)

print(current_gas_price, current_max_priority_gas, block_number, nonce)

#print()

tx = tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
                                                    src_address=flashloan_token_address, bnt_eth=bnt_eth,
                                                    expected_profit=Decimal('1000'), safety_override=True, result=tx_helpers.XS_TRANSACTION)

print(tx)

126848865631 88360001 17223359 202
build_tx= {'type': '0x2', 'maxFeePerGas': 126848865631, 'maxPriorityFeePerGas': 88360001, 'from': '0x329bD11A21d90a30F7Ced511dB8BBdCE28b81b36', 'nonce': 202}
{'value': 0, 'gas': 950000, 'chainId': 1, 'type': '0x2', 'maxFeePerGas': 126848865631, 'maxPriorityFeePerGas': 88360001, 'from': '0x329bD11A21d90a30F7Ced511dB8BBdCE28b81b36', 'nonce': 202, 'to': '0x41Eeba3355d7D6FF628B7982F3F9D055c39488cB', 'data': '0x55814b7c00000000000000000000000000000000000000000000000000000000000000600000000000000000000000001f573d6fb3f13d689ff844b4ce37794d79a7ff1c00000000000000000000000000000000000000000000003e1d3abfe8f196c51400000000000000000000000000000000000000000000000000000000000000030000000000000000000000000000000000000000000000000000000000000060000000000000000000000000000000000000000000000000000000000000018000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000a0b86991c62

In [25]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

build_tx= {'type': '0x2', 'maxFeePerGas': 126848865631, 'maxPriorityFeePerGas': 88360001, 'from': '0x329bD11A21d90a30F7Ced511dB8BBdCE28b81b36', 'nonce': 202}


2023-05-09 17:02:47,191 [fastlane:INFO] - gas estimate = 719901
2023-05-09 17:02:47,192 [fastlane:INFO] - Expected profit of 1000 BNT vs cost of 404.820224271810634754691634791900527948073676458511553865733552221656567340710 BNT in gas, executing
2023-05-09 17:02:47,194 [fastlane:INFO] - Attempting to submit private tx {'value': 0, 'gas': 694901, 'chainId': 1, 'type': '0x2', 'maxFeePerGas': 126848865631, 'maxPriorityFeePerGas': 88360001, 'from': '0x329bD11A21d90a30F7Ced511dB8BBdCE28b81b36', 'nonce': 202, 'to': '0x41Eeba3355d7D6FF628B7982F3F9D055c39488cB', 'data': '0x55814b7c00000000000000000000000000000000000000000000000000000000000000600000000000000000000000001f573d6fb3f13d689ff844b4ce37794d79a7ff1c00000000000000000000000000000000000000000000003e1d3abfe8f196c51400000000000000000000000000000000000000000000000000000000000000030000000000000000000000000000000000000000000000000000000000000060000000000000000000000000000000000000000000000000000000000000018000000000000000000000000000000000000

KeyboardInterrupt: 

# MAINNET Forced Tx Submission Method

In [ ]:
# # Initialize tx helper
# tx_helper = TxHelper(
#     usd_gas_limit=55,
#     ConfigObj=bot.ConfigObj,
#     gas_price_multiplier = 1
# )
# # # Submit the transaction
# # tx_helper.submit_flashloan_arb_tx(
# #     arb_data=route_struct,
# #     flashloan_token_address=flashloan_token_address,
# #     flashloan_amount=flashloan_amount,
# #     verbose=True,
# # )

In [ ]:
# tx_helper.gas_limit